# Boltzmann Machines

In [4]:
# pip install torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
# ------------------------------------------------------------
# Boltzmann Machines — Recommender System with PyTorch
# Project Overview
# Implemented a Restricted Boltzmann Machine (RBM) in PyTorch
# to predict user preferences from the MovieLens 100K dataset.
# Focused on collaborative filtering and unsupervised feature learning
# for binary and ternary rating transformations.
# ------------------------------------------------------------


In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [7]:
# ------------------------------------------------------------
# Data Loading & Preprocessing
# Step 1: Load MovieLens dataset (movies, users, ratings)
# Using Latin-1 encoding to handle metadata correctly.
# ------------------------------------------------------------

In [8]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [9]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [10]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [11]:
# ------------------------------------------------------------
# Data Formatting
# Step 2: Convert raw user–movie–rating triplets
# into structured tensors suitable for RBM training.
# Each row represents a user, and each column a movie.
# Ratings are normalized: -1 (unrated), 0 (neutral), 1 (liked)
# ------------------------------------------------------------


In [12]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [13]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [14]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [15]:
# ------------------------------------------------------------
# RBM Architecture
# Step 3: Define Restricted Boltzmann Machine (RBM) Class
# RBM uses visible (movie ratings) and hidden (latent features) layers.
# Implements stochastic binary sampling of hidden and visible units.
# Includes contrastive divergence (CD) weight updates.
# ------------------------------------------------------------


In [16]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0).t() - torch.mm(vk.t(), phk).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [17]:
# ------------------------------------------------------------
# Training Phase
# Step 4: Train the RBM using contrastive divergence (CD-10)
# Each epoch iterates over user batches to minimize reconstruction error.
# Training loss reflects prediction accuracy for rated items only.
# ------------------------------------------------------------


In [18]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3692)
epoch: 2 loss: tensor(0.2519)
epoch: 3 loss: tensor(0.2480)
epoch: 4 loss: tensor(0.2492)
epoch: 5 loss: tensor(0.2496)
epoch: 6 loss: tensor(0.2517)
epoch: 7 loss: tensor(0.2480)
epoch: 8 loss: tensor(0.2438)
epoch: 9 loss: tensor(0.2499)
epoch: 10 loss: tensor(0.2481)


In [19]:
# ------------------------------------------------------------
# Evaluation Phase
# Step 5: Evaluate on test set
# Compute mean absolute error between predicted and actual ratings.
# Demonstrates the RBM’s capacity to learn user–item relationships.
# ------------------------------------------------------------


In [20]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2620)
